In [2]:
from deap import base, creator, tools

import networkx as nx

import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from GraphColoringProblem import GraphColoringProblem

import sys
sys.path.insert(0, "C:/Jupyter/Evolution")
import elitysm

In [ ]:
POPULATION_SIZE = 100
P_CROSSOVER = 0.9
P_MUTATION = 0.1
MAX_GENERATIONS = 100
HALL_OF_FAME_SIZE = 5
HARD_CONSTRAINT_PENALTY = 10
MAX_COLORS = 10

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

toolbox = base.Toolbox()

gcp = GraphColoringProblem(nx.mycielski_graph(15), HARD_CONSTRAINT_PENALTY)

creator.create("FitnessMin", base.Fitness, weights = (-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("Integers", random.randint, 0, MAX_COLORS-1)
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.Integers, len(gcp))
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

def getCost(Individual):
    return gcp.getCost(Individual),

toolbox.register("evaluate", getCost)
toolbox.register("select", tools.selTournament, tournsize = 2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low = 0, up = MAX_COLORS, indpb = 1.0/len(gcp))


def main():
    population = toolbox.populationCreator(n = POPULATION_SIZE)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.average)
        
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
    
    population, logbook = elitysm.eaSimpleWithElitism(
        population, toolbox, cxpb= P_CROSSOVER,
        mutpb= P_MUTATION, ngen= MAX_GENERATIONS, 
        stats= stats, halloffame= hof, verbose= True)

     # print info for best solution found:
    best = hof.items[0]
    print("-- Best Individual = ", best)
    print("-- Best Fitness = ", best.fitness.values[0])
    print()
    print("number of colors = ", gcp.getNumberOfColors(best))
    print("Number of violations = ", gcp.getViolationsCount(best))
    print("Cost = ", gcp.getCost(best))

    # plot best solution:
    plt.figure(1)
    gcp.plotGraph(best)

    # extract statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

    # plot statistics:
    plt.figure(2)
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues, color='red')
    plt.plot(meanFitnessValues, color='green')
    plt.xlabel('Generation')
    plt.ylabel('Min / Average Fitness')
    plt.title('Min and Average fitness over Generations')

    plt.show()

if __name__=="__main__":
    main()